In [1]:
import json
import pandas as pd

with open('../panel_app/example_data/routing_tomtom_out.json') as f:
    response_json = json.load(f)
list_df_pts = []
leg_list= response_json['routes'][0]['legs']
for i in range(len(leg_list)):
    df = pd.DataFrame(leg_list[i]['points'])
    df['leg'] = i
    df['pt_num'] = np.arange(df.shape[0])
    list_df_pts.append(df)
df_all_pts = pd.concat(list_df_pts)

DEFAULT_DEST = [
    "1122 3 St SE",
    "1706 11 ave sw",
    "445 Marsh Rd NE",
    "504 Berkley Cres NW",
    "880 Center Ave NE",
    "111 25 Ave SW",
    "7016 34 Ave NW",
    "2313 15A st SE",
]

DEFAULT_LATLONG = [(51.04191, -114.05572),
 (51.04357, -114.0996),
 (51.05276, -114.05288),
 (51.12546, -114.08387),
 (51.05257, -114.04246),
 (51.03027, -114.06479),
 (51.08401, -114.18789),
 (51.03297, -114.02386)]



In [2]:
df_label = pd.DataFrame(data = DEFAULT_LATLONG,columns=['latitude','longitude'])
df_label['label'] = DEFAULT_DEST

In [3]:
import altair as alt
ch = alt.Chart(data = df_all_pts,width = 900,height=600).encode(x=alt.Longitude('longitude',scale=alt.Scale(zero=False)),
                                                                 y=alt.Latitude('latitude',scale=alt.Scale(zero=False)),color = 'leg:N' ).mark_point(filled=True)
ch_lab = alt.Chart(data=df_label,width = 900,height=600).encode(x=alt.Longitude('longitude',scale=alt.Scale(zero=False)),
                                                                 y=alt.Latitude('latitude',scale=alt.Scale(zero=False)),text = 'label:N' ).mark_text(size=15)

(ch+ch_lab).interactive()

alt.LayerChart(...)

In [4]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider,STAMEN_TERRAIN_RETINA,STAMEN_TERRAIN, Vendors
from bokeh.models import ColumnDataSource
import numpy as np
output_notebook()

Loading BokehJS ...

In [5]:
Vendors

Enumeration(CARTODBPOSITRON, CARTODBPOSITRON_RETINA, STAMEN_TERRAIN, STAMEN_TERRAIN_RETINA, STAMEN_TONER, STAMEN_TONER_BACKGROUND, STAMEN_TONER_LABELS)

In [6]:
from pyproj import Proj, transform

def reproject_coordinates(input_projection, output_projection, input_x, input_y, suffix=' Transformed'):
    """Reproject coordinate series
    Arguments:
        input_projection {string} -- Input Coordinate System descriptor (e.g. 'epsg:4267' for NAD27)
        output_projection {string} -- Output Coordinate System descriptor (e.g. 'epsg:26910' for NAD83)
        input_x {pandas.Series or list-like} -- Array of x values in input CRS
        input_y {pandas.Series or list-like} -- Array of y values in input CRS
    Keyword Arguments:
        suffix {str} -- Appended to original name of input x and y columns (default: {' Transformed'})
    Returns:
        pandas.DataFrame -- Dataframe with two columns: output_x and output_y
    """
    print(f'Converting from {input_projection} to {output_projection}')
    # The 'init=' generates a warning, but transform fails if it is not used!
    output_x, output_y = transform(Proj(init=input_projection, preserve_units=False), Proj(init=output_projection, preserve_units=False), input_x.to_list(), input_y.to_list())
    return pd.DataFrame(data={f'{input_x.name}{suffix}':output_x, f'{input_y.name}{suffix}':output_y})


In [7]:
new_coords=reproject_coordinates('epsg:4326','epsg:3857',df_all_pts['longitude'],df_all_pts['latitude'],suffix='_new')
df_all_pts['Longitude'] = new_coords['longitude_new'].values
df_all_pts['Latitude'] = new_coords['latitude_new'].values

df_source = ColumnDataSource(df_all_pts)

Converting from epsg:4326 to epsg:3857


/home/anton/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/anton/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/home/anton/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the pr

In [32]:
from bokeh.transform import CategoricalColorMapper        
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral10,Accent7, Dark2_5
import itertools
from bokeh.models import WMTSTileSource
def color_gen():
    yield from itertools.cycle(Dark2_5)
color = color_gen()

tile_options = {}
tile_options['url'] = 'https://a.tile.openstreetmap.org/{Z}/{X}/{Y}.png'
tile_options['attribution'] = """
'&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
    """
OSM_tile_source = WMTSTileSource(**tile_options)


df_all_pts['leg'] = df_all_pts['leg'].astype(str)
df_source = ColumnDataSource(df_all_pts)
unique_legs = df_all_pts['leg'].astype(str).unique().tolist()

cmap = factor_cmap('leg',palette = Spectral10,
                   factors = sorted(df_all_pts.leg.unique()))
# range bounds supplied in web mercator coordinates
xrange = (df_all_pts['Longitude'].round(decimals=2).min(),
          df_all_pts['Longitude'].round(decimals=2).max())
yrange = (df_all_pts['Latitude'].round(decimals=2).min(),
          df_all_pts['Latitude'].round(decimals=2).max())
p = figure(x_range=xrange,
           y_range=yrange,
            x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(OSM_tile_source)
#p.add_tile(OSM_tile_source)

#p.circle(x='Longitude',y ='Latitude',
          #fill_color = cmap,
#         color=cmap,
#          source =df_source)
# Add lines:
for l,c in zip(unique_legs,color):
    df_sub_source = ColumnDataSource(df_all_pts[df_all_pts['leg']==l])
    p.line(x='Longitude',y ='Latitude',
          #fill_color = cmap,
           color=c,
           source =df_sub_source,
           line_width=5,
           legend_label=f'Leg {l}'
      )

p.xaxis.axis_label = 'Longitude'
p.yaxis.axis_label = 'Latitude'

show(p)

In [28]:
response_json

{'formatVersion': '0.0.12',
 'routes': [{'summary': {'lengthInMeters': 60437,
    'travelTimeInSeconds': 5180,
    'trafficDelayInSeconds': 0,
    'departureTime': '2020-04-01T00:00:22-06:00',
    'arrivalTime': '2020-04-01T01:26:37-06:00',
    'noTrafficTravelTimeInSeconds': 5180,
    'historicTrafficTravelTimeInSeconds': 5180,
    'liveTrafficIncidentsTravelTimeInSeconds': 5180},
   'legs': [{'summary': {'lengthInMeters': 1299,
      'travelTimeInSeconds': 252,
      'trafficDelayInSeconds': 0,
      'departureTime': '2020-04-01T00:00:22-06:00',
      'arrivalTime': '2020-04-01T00:04:33-06:00',
      'noTrafficTravelTimeInSeconds': 252,
      'historicTrafficTravelTimeInSeconds': 252,
      'liveTrafficIncidentsTravelTimeInSeconds': 252},
     'points': [{'latitude': 51.04843, 'longitude': -114.06398},
      {'latitude': 51.04842, 'longitude': -114.0635},
      {'latitude': 51.0484, 'longitude': -114.06284},
      {'latitude': 51.04834, 'longitude': -114.06035},
      {'latitude': 51

In [44]:
routes =response_json['routes'][0]
for i in routes['guidance']['instructions']:
    if i['instructionType'] == 'LOCATION_WAYPOINT':
        print(i['point'])

{'latitude': 51.04192, 'longitude': -114.0559}
{'latitude': 51.03297, 'longitude': -114.02364}
{'latitude': 51.03039, 'longitude': -114.06478}
{'latitude': 51.04341, 'longitude': -114.09961}
{'latitude': 51.08388, 'longitude': -114.18788}
{'latitude': 51.12559, 'longitude': -114.08382}
{'latitude': 51.05243, 'longitude': -114.04247}
{'latitude': 51.05288, 'longitude': -114.05288}


{'summary': {'lengthInMeters': 60437,
  'travelTimeInSeconds': 5180,
  'trafficDelayInSeconds': 0,
  'departureTime': '2020-04-01T00:00:22-06:00',
  'arrivalTime': '2020-04-01T01:26:37-06:00',
  'noTrafficTravelTimeInSeconds': 5180,
  'historicTrafficTravelTimeInSeconds': 5180,
  'liveTrafficIncidentsTravelTimeInSeconds': 5180},
 'legs': [{'summary': {'lengthInMeters': 1299,
    'travelTimeInSeconds': 252,
    'trafficDelayInSeconds': 0,
    'departureTime': '2020-04-01T00:00:22-06:00',
    'arrivalTime': '2020-04-01T00:04:33-06:00',
    'noTrafficTravelTimeInSeconds': 252,
    'historicTrafficTravelTimeInSeconds': 252,
    'liveTrafficIncidentsTravelTimeInSeconds': 252},
   'points': [{'latitude': 51.04843, 'longitude': -114.06398},
    {'latitude': 51.04842, 'longitude': -114.0635},
    {'latitude': 51.0484, 'longitude': -114.06284},
    {'latitude': 51.04834, 'longitude': -114.06035},
    {'latitude': 51.04738, 'longitude': -114.06042},
    {'latitude': 51.04692, 'longitude': -114.0